#  Extracting data on Sold properties with Domain api

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Roudra Das    roudra.das@gmail.com                                            |
| External References   | <a href="https://api.domain.com.au/" target="_blank">Domain API</a>|
| Input Datasets        |  List For Sale |
| Output Datasets       |  Table|
| Input Data Source     |  API |
| Output Data Source    | Pandas Dataframe |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 15th May 2021 | Roudra Das | Notebook created to extract properties that are on the market in Australia|

## Other Details
This Notebook is a prototype.

In [75]:

import pandas as pd
import requests
import json
import re, string, timeit
import time


In [76]:
pd.options.display.max_columns = None # show all columns in display

## Functions

In [77]:
def get_api_key(api_key_id = "Domain"):
  """
  Get the api key for website accessing.

  Table of key type and key value for privacy.

  Parameters
  ----------
  @api_key_id [string]: Key value in dataframe

  Returns
  -------
  [string]: client_id & client_secret

  """
  # load api keys file
  df_api_keys = pd.read_csv('~/Documents/Python/api_keys.csv', header = 'infer')
  
  # return api key if in dataset
  try:
    # get api key from id
    client_id = df_api_keys.loc[df_api_keys['Id'] == api_key_id]['Client'].iloc[0] # get client by id
    client_secret = df_api_keys.loc[df_api_keys['Id'] == api_key_id]['Secret'].iloc[0] # get secret by id
    # return api key
    return client_id, client_secret
  except IndexError:
    # get api key id list
    api_key_id_list = df_api_keys['Id'].unique().tolist()
    # print error message
    print('Cannot map key. Api key id must be one of the following options {0}'.format(api_key_id_list))

In [78]:
def api_property_list_for_sale(auth, property_type, bedrooms, bathrooms, suburb, postcode):
  # url for api
  url = "https://api.domain.com.au/v1/listings/residential/_search"

  # enter parameters
  post_fields ={
      "listingType":"Sale",
        "maxPrice":"",
        "pageSize":200,
      "propertyTypes":property_type,
      "minBedrooms":"",
        "maxBedrooms":"",
      "minBathrooms":"",
        "maxBathrooms":"",
      "locations":[
        {
          "state":"",
          "region":"",
          "area":"",
          "suburb":suburb,
          "postCode":postcode,
          "includeSurroundingSuburbs":False
        }
      ]
    }

  # response
  response = requests.post(url,headers=auth,json=post_fields)
  #response = requests.request("GET", url, headers=headers, params=querystring)
  return response.json()

In [79]:
def process_list_for_sale_response(response_json):
    """
    Process the list for sale API response.

    Convert each listing to a dataframe, append to a list, and concatenate to one dataframe.

    Parameters
    ----------
    @response_json [dictionary]: API response for list for sale

    Returns
    -------
    [dataframe] Dataframe of all list for sale responses

    """

    # empty dataframe
    dataframe_list = []

    # iterate through each for sale listing
    for j in range(len(response_json)):
        #response_json[j]['listing']['propertyDetails'].pop('features')
        k = response_json[j]['listing']['propertyDetails'].copy()
        k['id']  = response_json[j]['listing']['id']
        print(k)
        # convert each listing to dataframe 
        _temp_df = pd.DataFrame.from_dict(k, orient='index').T

        # append to dataframe list for all listings
        dataframe_list.append(_temp_df)
    
        # concatenate all dataframes, for missing col values enter null value
    return pd.concat(dataframe_list, axis=0, ignore_index=True, sort=False)

In [80]:
# setup
property_id="2016858650"
starting_max_price=100000
increment=50000
# when starting min price is zero we'll just use the lower bound plus 400k later on
starting_min_price=0

In [81]:
# POST request for token
client_id, client_secret = get_api_key(api_key_id="Domain")
response = requests.post('https://auth.domain.com.au/v1/connect/token', data = {'client_id':client_id,"client_secret":client_secret,"grant_type":"client_credentials","scope":"api_listings_read","Content-Type":"text/json"})
token=response.json()
access_token=token["access_token"]

In [82]:
#domain_api_key = get_api_key(api_key_id = "Domain")
# GET Request for ID
url = "https://api.domain.com.au/v1/listings/"+property_id
auth = {"Authorization":"Bearer "+access_token}
request = requests.get(url,headers=auth)
r=request.json()

In [83]:
#get details
da=r['addressParts']
postcode=da['postcode']
suburb=da['suburb']
bathrooms=r['bathrooms']
bedrooms=r['bedrooms']
carspaces=r['carspaces']
property_type=r['propertyTypes']
area=r['landAreaSqm']
geolocation=r['geoLocation']

print(property_type, postcode, suburb, bedrooms, bathrooms,  carspaces, area, geolocation)

# the below puts all relevant property types into a single string. eg. a property listing can be a 'house' and a 'townhouse'
n=0
property_type_str=""
for p in r['propertyTypes']:
  property_type_str=property_type_str+(r['propertyTypes'][int(n)])
  n=n+1
print(property_type_str) 

['house'] 2641 Lavington 4.0 2.0 2.0 711.0 {'latitude': -36.0284049, 'longitude': 146.94486}
house


In [84]:

property_list_for_sale_response = api_property_list_for_sale(auth, property_type, bedrooms, bathrooms, suburb, postcode)
property_list_for_sale_response[:5]

[{'type': 'PropertyListing',
  'listing': {'listingType': 'Sale',
   'id': 2016990302,
   'advertiser': {'type': 'Agency',
    'id': 11631,
    'name': 'Stean Nicholls Pty Ltd',
    'logoUrl': 'https://images.domain.com.au/img/Agencys/11631/logo_11631.GIF',
    'preferredColourHex': '#000033',
    'bannerUrl': 'https://images.domain.com.au/img/Agencys/11631/banner_11631.GIF',
    'contacts': [{'name': 'Jack Stean',
      'photoUrl': 'https://images.domain.com.au/img/11631/contact_1400351.jpeg?mod=210514-135226'},
     {'name': 'Mikaela Gould',
      'photoUrl': 'https://images.domain.com.au/img/11631/contact_1794690.jpeg?mod=210510-145920'}]},
   'priceDetails': {'displayPrice': 'Auction Saturday 29th May at 10am'},
   'media': [{'category': 'Image',
     'url': 'https://bucket-api.domain.com.au/v1/bucket/image/2016990302_1_1_210507_043526-w2500-h1667'},
    {'category': 'Image',
     'url': 'https://bucket-api.domain.com.au/v1/bucket/image/2016990302_2_1_210507_043526-w2500-h1667'},
 

In [85]:
df_properties_for_sale_raw = process_list_for_sale_response(response_json=property_list_for_sale_response)

df_properties_for_sale_raw

{'state': 'NSW', 'features': ['BuiltInWardrobes', 'Ensuite', 'Heating', 'Study', 'Shed'], 'propertyType': 'House', 'allPropertyTypes': ['House'], 'bathrooms': 3.0, 'bedrooms': 8.0, 'carspaces': 2, 'unitNumber': '', 'streetNumber': '453', 'street': 'Dale Crescent', 'area': 'Albury - Greater Region', 'region': 'Regional NSW', 'suburb': 'LAVINGTON', 'postcode': '2641', 'displayableAddress': '453 Dale Crescent, Lavington', 'latitude': -36.0435524, 'longitude': 146.949036, 'landArea': 910.0, 'id': 2016990302}
{'state': 'NSW', 'features': ['BuiltInWardrobes', 'Gas', 'SwimmingPool', 'Heating', 'Shed'], 'propertyType': 'House', 'allPropertyTypes': ['House'], 'bathrooms': 2.0, 'bedrooms': 4.0, 'carspaces': 2, 'unitNumber': '', 'streetNumber': '388', 'street': 'Robert Crt', 'area': 'Albury - Greater Region', 'region': 'Regional NSW', 'suburb': 'LAVINGTON', 'postcode': '2641', 'displayableAddress': '388 Robert Crt, Lavington', 'latitude': -36.0467453, 'longitude': 146.935226, 'landArea': 912.0, '

,state,features,propertyType,allPropertyTypes,bathrooms,bedrooms,carspaces,unitNumber,streetNumber,street,area,region,suburb,postcode,displayableAddress,latitude,longitude,landArea,id,buildingArea
0,NSW,"[BuiltInWardrobes, Ensuite, Heating, Study, Shed]",House,[House],3,8,2,,453,Dale Crescent,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"453 Dale Crescent, Lavington",-36.0436,146.949,910,2016990302,NaN
1,NSW,"[BuiltInWardrobes, Gas, SwimmingPool, Heating,...",House,[House],2,4,2,,388,Robert Crt,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"388 Robert Crt, Lavington",-36.0467,146.935,912,2016987771,NaN
2,NSW,"[Ensuite, SecureParking, Heating, Shed, SolarP...",House,[House],2,4,2,,21,Valerie Way,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"21 Valerie Way, Lavington",-36.0343,146.942,799,2016986063,NaN
3,NSW,"[Ensuite, SecureParking, Shed]",House,[House],2,3,2,,53,Lawson Circuit,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"53 Lawson Circuit, Lavington",-36.0435,146.957,631,2016984656,NaN
4,NSW,"[AirConditioning, BuiltInWardrobes, Gas, Swimm...",House,[House],1,2,1,31,7,Catherine Crescent,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"31/7 Catherine Crescent, Lavington",-36.0411,146.958,NaN,2016983014,NaN
5,NSW,"[AirConditioning, BuiltInWardrobes, Ensuite, G...",House,[House],2,3,2,,12,Brookfields Mews,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"12 Brookfields Mews, Lavington",-36.0457,146.926,359,2016978657,NaN
6,NSW,"[AirConditioning, Gas, Heating, Shed]",House,[House],1,3,2,,497,Parnall Street,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"497 Parnall Street, Lavington",-36.0387,146.931,780,2016969545,NaN
7,NSW,"[Ensuite, Floorboards, SecureParking, Heating,...",House,[House],3,3,2,,283,Highview Crescent,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"283 Highview Crescent, Lavington",-36.0469,146.953,847,2016962604,NaN
8,NSW,"[AirConditioning, BuiltInWardrobes, Gas, Secur...",House,[House],2,4,4,,496,Prune St,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"496 Prune St, Lavington",-36.0394,146.936,923,2016955601,NaN
9,NSW,"[AirConditioning, BuiltInWardrobes, Gas, Secur...",House,[House],1,3,4,,671,Pearsall Street,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"671 Pearsall Street, Lavington",-36.0378,146.921,976,2016949385,NaN


In [86]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}"">{}</a>'.format(val, val)

In [87]:
df_properties_for_sale_raw.shape

(36, 20)

In [88]:
df_properties_for_sale_raw.to_csv("output/ForSale_Properties.csv", index = False)

In [90]:
from sklearn.cluster import DBSCAN


df_properties_for_sale_raw["labels"] = DBSCAN(eps=0.01, min_samples=3).fit(df_properties_for_sale_raw[["latitude","longitude"]].values).labels_
df_properties_for_sale_raw = df_properties_for_sale_raw.drop(['allPropertyTypes','buildingArea'], axis = 1)
df_properties_for_sale_raw = df_properties_for_sale_raw.dropna()
df_properties_for_sale_raw.landArea = df_properties_for_sale_raw.landArea.astype(int)
df_properties_for_sale_raw.bathrooms = df_properties_for_sale_raw.bathrooms.astype(int)
df_properties_for_sale_raw.bedrooms = df_properties_for_sale_raw.bedrooms.astype(int)
df_properties_for_sale_raw.carspaces = df_properties_for_sale_raw.carspaces.astype(int)
df_properties_for_sale_raw['URL'] = 'http://www.domain.com.au/' + df_properties_for_sale_raw.id.apply(str)
df_properties_for_sale_raw = df_properties_for_sale_raw.drop(df_properties_for_sale_raw[df_properties_for_sale_raw["landArea"] > 2000].index)
df_properties_for_sale = df_properties_for_sale_raw.style.format({'URL': make_clickable})

In [94]:
df_properties_for_sale

,state,features,propertyType,bathrooms,bedrooms,carspaces,unitNumber,streetNumber,street,area,region,suburb,postcode,displayableAddress,latitude,longitude,landArea,id,labels,URL
0,NSW,"['BuiltInWardrobes', 'Ensuite', 'Heating', 'Study', 'Shed']",House,3,8,2,,453,Dale Crescent,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"453 Dale Crescent, Lavington",-36.043552,146.949036,910,2016990302,0,http://www.domain.com.au/2016990302
1,NSW,"['BuiltInWardrobes', 'Gas', 'SwimmingPool', 'Heating', 'Shed']",House,2,4,2,,388,Robert Crt,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"388 Robert Crt, Lavington",-36.046745,146.935226,912,2016987771,0,http://www.domain.com.au/2016987771
2,NSW,"['Ensuite', 'SecureParking', 'Heating', 'Shed', 'SolarPanels']",House,2,4,2,,21,Valerie Way,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"21 Valerie Way, Lavington",-36.034313,146.941900,799,2016986063,0,http://www.domain.com.au/2016986063
3,NSW,"['Ensuite', 'SecureParking', 'Shed']",House,2,3,2,,53,Lawson Circuit,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"53 Lawson Circuit, Lavington",-36.043500,146.956757,631,2016984656,0,http://www.domain.com.au/2016984656
5,NSW,"['AirConditioning', 'BuiltInWardrobes', 'Ensuite', 'Gas', 'SecureParking', 'Heating']",House,2,3,2,,12,Brookfields Mews,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"12 Brookfields Mews, Lavington",-36.045715,146.925980,359,2016978657,0,http://www.domain.com.au/2016978657
6,NSW,"['AirConditioning', 'Gas', 'Heating', 'Shed']",House,1,3,2,,497,Parnall Street,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"497 Parnall Street, Lavington",-36.038746,146.930634,780,2016969545,0,http://www.domain.com.au/2016969545
7,NSW,"['Ensuite', 'Floorboards', 'SecureParking', 'Heating', 'Study']",House,3,3,2,,283,Highview Crescent,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"283 Highview Crescent, Lavington",-36.046932,146.952728,847,2016962604,0,http://www.domain.com.au/2016962604
8,NSW,"['AirConditioning', 'BuiltInWardrobes', 'Gas', 'SecureParking', 'Heating', 'Study', 'Shed', 'FullyFenced']",House,2,4,4,,496,Prune St,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"496 Prune St, Lavington",-36.039400,146.935760,923,2016955601,0,http://www.domain.com.au/2016955601
9,NSW,"['AirConditioning', 'BuiltInWardrobes', 'Gas', 'SecureParking', 'AlarmSystem', 'Heating', 'Dishwasher', 'Shed']",House,1,3,4,,671,Pearsall Street,Albury - Greater Region,Regional NSW,LAVINGTON,2641,"671 Pearsall Street, Lavington",-36.037823,146.921051,976,2016949385,0,http://www.domain.com.au/2016949385
10,NSW,"['BuiltInWardrobes', 'Gas', 'SecureParking', 'Heating', 'Dishwasher', 'Shed', 'FullyFenced']",House,2,3,3,,27,Mountain View Dr (Quicks Hill),Albury - Greater Region,Regional NSW,LAVINGTON,2641,"27 Mountain View Dr (Quicks Hill), Lavington",-36.049522,146.953949,683,2016944320,0,http://www.domain.com.au/2016944320


In [91]:
def search_for_price(data, starting_min_price, starting_max_price, increment):
    
    url = "https://api.domain.com.au/v1/listings/residential/_search" # Set destination URL here

    max_price=starting_max_price

    searching_for_price_l = True
    while searching_for_price_l:
        post_fields ={
        "listingType":"Sale",
            "maxPrice":max_price,
            "pageSize":100,
        "propertyTypes":['house'],
        "minBedrooms":data['bedrooms'],
            "maxBedrooms":data['bedrooms'],
        "minBathrooms":data['bathrooms'],
            "maxBathrooms":data['bathrooms'],
        "locations":[
            {
            "state":"",
            "region":"",
            "area":"",
            "suburb":data['suburb'],
            "postCode":data['postcode'],
            "includeSurroundingSuburbs":False
            }
        ]
        }

        request = requests.post(url,headers=auth,json=post_fields)

        l=request.json()
        listings = []
        for listing in l:
            listings.append(listing["listing"]["id"])
        listings

        if int(property_id) in listings:
                max_price=max_price-increment
                print("Lower bound found: ", max_price)
                searching_for_price_l=False
        else:
            max_price=max_price+increment
            print("Not found. Increasing max price to ",max_price)
            time.sleep(0.1)  # sleep a bit so you don't make too many API calls too quickly )


    if starting_min_price>0:
            min_price=starting_min_price
            
    else:  
            min_price=max_price+400000


    searching_for_price_u = True
    while searching_for_price_u:
        post_fields ={
        "listingType":"Sale",
            "minPrice":min_price,
            "pageSize":100,
        "propertyTypes":['house'],
        "minBedrooms":data['bedrooms'],
            "maxBedrooms":data['bedrooms'],
        "minBathrooms":data['bathrooms'],
            "maxBathrooms":data['bathrooms'],
        "locations":[
            {
            "state":"",
            "region":"",
            "area":"",
            "suburb":data['suburb'],
            "postCode":data['postcode'],
            "includeSurroundingSuburbs":False
            }
        ]
        }

        request = requests.post(url,headers=auth,json=post_fields)

        l=request.json()
        listings = []
        for listing in l:
            listings.append(listing["listing"]["id"])
        listings

        if int(property_id) in listings:
                min_price=min_price+increment
                print("Upper bound found: ", min_price)
                searching_for_price_u=False
        else:
            min_price=min_price-increment
            print("Not found. Decreasing min price to ",min_price)
            time.sleep(0.1)  # sleep a bit so you don't make too many API calls too quickly )

        if max_price<1000000:
            lower=max_price/1000
            upper=min_price/1000
            denom="k"
        else: 
            lower=max_price/1000000
            upper=min_price/1000000
            denom="m"
    

    return print("Price range:","$",lower,"-","$",upper,denom)

In [92]:
import plotly.express as px

fig = px.scatter_mapbox(df_properties_for_sale_raw, lat="latitude", lon="longitude", size = "landArea", hover_name="URL", hover_data=["propertyType", "bedrooms", "bathrooms", "landArea", "displayableAddress"],zoom=15, height=600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# End of Notebook